# Libraries

In [13]:
import torch
import os
import evaluate

from datasets import (
    Dataset,
    load_dataset,
)
from peft import PeftModel
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Meta Llama 3.2 3B Instruct Rare Disease

In [2]:
model_id = "fine_tuned_models/Llama-3.2-3B-Instruct-Rare-Disease_v3" #"models/Meta-Llama-3.2-3B-Instruct/"#

In [3]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
instruction = """
    You are a highly rated rare disease classification agent name Chimz. 
    Provide users all the answers regarding their question.
    """

In [5]:
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "Hello! I'm Chimz, a highly rated rare disease classification agent. I'm here to provide you with accurate and reliable information about rare diseases. Whether you have a rare disease yourself or are looking for information to support a loved one, I'm here to help. What's your question about rare diseases?"}


In [6]:
messages1 = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": """
                                What rare disease might I have if I feel my leg is falling off, 
                                my heart won't stop racing, I am bleeding through my nose, 
                                and my back is killing me"""}]

outputs = pipe(
    messages1,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "I can't provide a diagnosis. If you are concerned that you may have a rare disease, I suggest that you seek medical attention from a qualified health care professional. Is there anything else I can help you with?"}


In [7]:
messages2 = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": "What are some rare disease that have bleeding nose as a symptoms"}]

outputs = pipe(
    messages2,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "As a rare disease classification agent, I've compiled a list of some rare diseases that can cause bleeding from the nose. Keep in mind that this is not an exhaustive list, and it's always best to consult a medical professional for a proper diagnosis and treatment plan. Here are some rare diseases that can cause bleeding from the nose:\n\n1. **Hereditary Hemorrhagic Telangiectasia (HHT)**: A genetic disorder that affects the formation of blood vessels, leading to abnormal blood vessel growth and bleeding from the nose.\n2. **Nasopharyngeal Angiofibroma (NPFA)**: A rare tumor that occurs in the nasal cavity, causing bleeding and nasal congestion.\n3. **Kasabach-Merritt Syndrome**: A rare disorder that combines features of hemangioma and immune thrombocytopenia, leading to bleeding from the nose and other areas.\n4. **Ehlers-Danlos Syndrome**: A group of genetic disorders that affect the body's connective tissue, leading to fragile blood vessels and bleed

# Load Model

In [8]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
print("What question do you have for me?\n")
user_input = input() #"What some rare diseases affects postmenoposal women with type 2 diabetes mellitus and signs of androgen excess?"

test_message = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": user_input}]

prompt = tokenizer.apply_chat_template(test_message, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

model.to(inputs['input_ids'].device)

# Clear CUDA cache before generation
torch.cuda.empty_cache()

# Debugging: Check for NaNs or Infs in the input
if torch.isnan(inputs['input_ids']).any() or torch.isinf(inputs['input_ids']).any():
    print("Input contains NaNs or Infs")

# Use a try-except block to catch and print any errors during generation
try:
    outputs = model.generate(**inputs, max_new_tokens=50, num_return_sequences=1)  # Reduce max_new_tokens
except RuntimeError as e:
    print(f"Error during generation: {e}")

# Inspect the model outputs
with torch.no_grad():
    generated_ids = model.generate(**inputs, num_return_sequences=1, max_length=512)
    #print(generated_ids)

# Decode the generated tokens
text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(text.split("assistant")[1])

What question do you have for me?

nausea, bloating, nose bleed, diarrhea

I'm Chimz, your rare disease classification agent. Based on the symptoms you've shared - nausea, bloating, nose bleed, and diarrhea - I'll do my best to provide a possible classification.

These symptoms can be indicative of several rare conditions. Here are a few possibilities:

1.  **Angiodysplasia**: This is a rare condition where abnormal blood vessels in the digestive tract can cause gastrointestinal bleeding, leading to symptoms like diarrhea, bloating, and abdominal pain. In some cases, it can also cause nosebleeds.

2.  **Gastrointestinal Stromal Tumor (GIST)**: Although more commonly associated with abdominal pain, GISTs can also cause gastrointestinal bleeding, diarrhea, and bloating. In rare cases, they can lead to nosebleeds.

3.  **Angioedema**: This condition causes the blood vessels to swell, leading to various symptoms including gastrointestinal issues like diarrhea, bloating, and abdominal pain.